## Question 1

Now convert this model from Keras to TF-Lite format.

https://github.com/alexeygrigorev/large-datasets/releases/download/wasps-bees/bees-wasps.h5

What's the size of the **converted** model?

* 21 Mb
* **43 Mb**
* 80 Mb
* 164 Mb

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpbrzbtdeo/assets


INFO:tensorflow:Assets written to: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpbrzbtdeo/assets
2023-11-26 20:48:08.003453: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-11-26 20:48:08.003481: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-11-26 20:48:08.003653: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpbrzbtdeo
2023-11-26 20:48:08.004958: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-26 20:48:08.004967: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/m5/90bb4mps3dv507n82593klgr0000gn/T/tmpbrzbtdeo
2023-11-26 20:48:08.007149: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-11-26 20:48:08.007783: I tensorflow/cc/saved_model/load

In [3]:
!ls -lh

total 262984
-rw-r--r--@ 1 koda  staff    86M Nov 25 15:51 bees-wasps.h5
-rw-r--r--@ 1 koda  staff    43M Nov 26 20:48 bees-wasps.tflite
-rw-r--r--@ 1 koda  staff    21K Nov 26 20:48 hw9.ipynb


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

* 3
* 7
* **13**
* 24

In [4]:
import tensorflow.lite as tflite

In [35]:
interpreter = tflite.Interpreter(model_path="bees-wasps.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
output_index

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


13

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

```python
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img
```

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

Based on the previous homework, what should be the target size for the image?

In [6]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [7]:
image = download_image("https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg")
image = prepare_image(image, (150, 150))

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3450980
* 0.5450980
* 0.7450980
* **0.9450980**

In [9]:
import numpy as np

In [29]:
def rescale_image(x):
    return x / 255

In [37]:
x = np.array(image, dtype=np.float32)
X = np.array([x])

X = rescale_image(X)

In [38]:
X[0, 0, 0, 0]

0.94509804

## Question 4

Now let's apply this model to this image. What's the output of the model?

* 0.258
* 0.458
* **0.658**
* 0.858

In [39]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)

In [45]:
preds[0, 0]

0.6589834

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

* 162 Mb
* 362 Mb
* **662 Mb**
* 962 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

In [46]:
!docker images

REPOSITORY                      TAG         IMAGE ID       CREATED       SIZE
agrigorev/zoomcamp-bees-wasps   v2          b9f6c13de368   9 days ago    662MB
owykj0                          e6mfa9      219205aee4c5   2 weeks ago   727MB
smoker-prediction               latest      219205aee4c5   2 weeks ago   727MB
8yiq8m                          5u6wr9      9e0765bec84f   2 weeks ago   727MB
python                          3.10-slim   9527ed6cf78d   6 weeks ago   154MB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference). 
The provided model requires the same preprocessing for images regarding target size and rescaling the value range than used in homework 8.

Now run the container locally.

Score this image: https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

* 0.2453
* **0.4453**
* 0.6453
* 0.8453

In [48]:
!python test.py

0.44533517956733704


## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.


## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -t zoomcamp-bees-wasps -f homework.dockerfile .
docker tag zoomcamp-bees-wasps:latest agrigorev/zoomcamp-bees-wasps:v2
docker push agrigorev/zoomcamp-bees-wasps:v2
```